In [3]:
SOURCE_FOLDER_JAP = "demo_jap"
SOURCE_FOLDER_TRANSLATION = "demo_rus"
FOLDER_TO_PATCH = "jap_3_01"
TRANSLATION_FOLDER = "translation_files"

from dataclasses import dataclass
import difflib
import itertools
import os
from game_types import Actor, Armor, Class, Enemy, Item, State, Weapon, Skill, str_repr
import shutil
from pathlib import Path

In [ ]:
# MAPS_FOLDER = "Maps"   

# old_maps = {}
# for item in Path(f"{SOURCE_FOLDER_JAP}/{MAPS_FOLDER}").glob('*.txt'):
#     with open(item, 'r', encoding='utf-8') as old_file:
#         old_maps[item] = old_file.read()

# new_maps = {}
# for item in Path(f"{FOLDER_TO_PATCH}/{MAPS_FOLDER}").glob('*.txt'):
#     with open(item, 'r', encoding='utf-8') as new_file:
#         new_maps[item] = new_file.read()
# from multiprocessing.pool import ThreadPool
# from threading import Lock

# PYTHON_GIL=0

# id_map = {}
# id_map_lock = Lock()
# mapped_maps = []
# mapped_maps_lock = Lock()

# def calculate_simularity(old_map):
#     print(old_map)
#     score = []
#     for new_map in new_maps.keys():
#         with mapped_maps_lock:
#             if new_map in mapped_maps:
#                 continue
#         simularity_score = difflib.SequenceMatcher(None, old_maps[old_map], new_maps[new_map]).ratio()
#         score.append((new_map, simularity_score))
#         if simularity_score > 0.9:
#             break

#     score.sort(key=lambda x: x[1], reverse=True)
#     if len(score) > 0:
#         with id_map_lock:
#             id_map[old_map] = (score[0][0], score[0][1])
#         with mapped_maps_lock:
#             mapped_maps.append(score[0][0])

# pool = ThreadPool(100)
# pool.map(calculate_simularity, old_maps.keys())

# import re
# rule = re.compile(r".*([一-龯]|[ぁ-んァ-ン]).*\s")
# ignored_words = ["Label", "Name", "When", "CommentMore", "Comment", "ChangeBattleBack", "ShowScrollingText", "ConditionalBranch"]
# useful_words = ["ShowText(", "Display Name", "ShowChoices", "display_skill_name", "ScriptMore", "unlimited_choices", "ex_choice_add"]
# japanese_pattern = re.compile(r'[\u3040-\u309F\u30A0-\u30FF\u4E00-\u9FAF]')
# for key in old_maps.keys():
#     for line in old_maps[key].split("\n"):
#         if japanese_pattern.search(line):
#             its_ok = False
#             for word in ignored_words:
#                 if word in line:
#                     its_ok = True
#             for word in useful_words:
#                 if word in line:
#                     its_ok = True
#             if not its_ok:
#                 print(line)


In [4]:
MAPS_FOLDER = "Maps"   
useful_words = ["ShowText(", "Display Name", "ShowChoices", "display_skill_name", "ScriptMore", "unlimited_choices", "ex_choice_add", "When"]

def keep_line(line):
    for word in useful_words:
        if word in line:
            return True
    return False

old_maps = {}
for item in Path(f"{SOURCE_FOLDER_JAP}/{MAPS_FOLDER}").glob('**/*.txt'):
    with open(item, 'r', encoding='utf-8') as old_file:
        temp = [x for x in old_file.readlines() if keep_line(x)]
        if temp:
            old_maps[item] = set(temp)

new_maps = {}
for item in Path(f"{FOLDER_TO_PATCH}/{MAPS_FOLDER}").glob('**/*.txt'):
    if os.path.isfile(item):
        with open(item, 'r', encoding='utf-8') as new_file:
            temp = [x for x in new_file.readlines() if keep_line(x)]
            if temp:
                new_maps[item] = set(temp)



In [35]:
from copy import deepcopy
id_map = {}
new_map_copy = deepcopy(new_maps)

for old_map in old_maps.keys():
    score = []
    for new_map in new_map_copy.keys():
        simularity_score = len(old_maps[old_map] & new_maps[new_map]) / len(new_maps[new_map])
        score.append((new_map, simularity_score))


    score.sort(key=lambda x: x[1], reverse=True)
    # print(score)
    # if len(score) > 0: 
    #     id_map[old_map] = (score[0][0], score[0][1])

In [7]:
total_unchanged_maps = 0
for key in id_map.keys():
        if id_map[key][1] == 1.0:
            total_unchanged_maps += 1
            continue
        print(key, id_map[key])
print(f"Total unchanged maps: {total_unchanged_maps}")
print(len(new_maps))

In [22]:
import re
vocabulary = {}
simple_vocabulary = {}
japanese_pattern = re.compile(r'[\u3040-\u309F\u30A0-\u30FF\u4E00-\u9FAF]')
# …|？|■|O|K|。|～|♪|○|×|△|□|◇|※|☆|◇|！|―|【|￣

def is_end_of_chunk(text, inx, context_length):
    for i in range(inx, min(len(text), inx + context_length + 1)):
        if keep_line(text[i]):
            return False
    return True


def break_text_into_chunks(text, context_length=3):
    chunks = []
    text = text.split("\n")
    i = 0
    while i < len(text):
        if keep_line(text[i]):
            start_of_chunk = i
            j = i + 1
            while not is_end_of_chunk(text, j, context_length):
                j += 1
            chunks.append(("\n".join(text[i-context_length:start_of_chunk]), "\n".join(text[start_of_chunk:j]), "\n".join(text[j:j+context_length])))
            i = j
        else:
            i += 1
    return chunks
            
def break_chunk(text):
    text = text.split("\n")
    first = 0
    last = 0
    while last < len(text):
        if not keep_line(text[last]):
            yield "\n".join(text[first:last])
            while last < len(text) and not keep_line(text[last]):
                last += 1
            first = last
            last += 1
        else:
            last += 1
    yield "\n".join(text[first:last])

# def try_to_autofind_position(text, context, previous_position,left=True):
#     context = context.split("\n")
#     context_pieces = len(context)
#     context_pieces_founded = 0
#     for piece in context:
#         if text.find(piece,previous_position-20) != -1:
#             context_pieces_founded += 1
#     print(f"founded {context_pieces_founded} / {context_pieces}")
#     return -1

def process_chunks(chunks, translated_text):
    context_was_not_founded = False
    last_context_position = 0
    for chunk in chunks:
        start_position = translated_text.find(chunk[0], last_context_position)
        if start_position != -1:
            start_position += len(chunk[0])
        end_position = translated_text.find(chunk[2], start_position)           
        if start_position != -1 and end_position != -1:
            for source, target in zip(break_chunk(chunk[1]), break_chunk(translated_text[start_position+1: end_position-1])):
                vocabulary[source] = target
        else:
            context_was_not_founded = True

        last_context_position = start_position
    return context_was_not_founded

def create_simple_shunks(text):
    chunks = []
    lines = text.split("\n")
    start_position = 0
    end_position = 0
    i = 0
    while i < len(lines):
        if keep_line(lines[i]):
            start_position = i
            end_position = i + 1
            while end_position < len(lines) and keep_line(lines[end_position]):
                end_position += 1
            chunks.append("\n".join(lines[start_position:end_position]))
            i = end_position + 1
        else:
            i += 1
    return chunks
        

for item in Path(f"{SOURCE_FOLDER_JAP}/{MAPS_FOLDER}").glob('**/*.txt'):
    print(item)
    source_text = ""
    translated_text = ""
    with open(item, 'r', encoding='utf-8') as source_file:
        source_text = source_file.read()
    with open(str(item).replace(SOURCE_FOLDER_JAP, SOURCE_FOLDER_TRANSLATION), 'r', encoding='utf-8') as translate_file:
        translated_text = translate_file.read()
    chunks = break_text_into_chunks(source_text)
    context_errored = process_chunks(chunks, translated_text)

    jap_simple_chunks = create_simple_shunks(source_text)
    rus_simple_chunks = create_simple_shunks(translated_text)
    if len(jap_simple_chunks) == len(rus_simple_chunks):
        for jap, rus in zip(jap_simple_chunks, rus_simple_chunks):
            simple_vocabulary[jap] = rus

    if len(jap_simple_chunks) != len(rus_simple_chunks) and context_errored:
        print(f"nigger {len(jap_simple_chunks)} \n {len(rus_simple_chunks)}")

print(len(simple_vocabulary))
print(len(vocabulary))



demo_jap\Maps\Map001.txt
demo_jap\Maps\Map002.txt
demo_jap\Maps\Map003.txt
demo_jap\Maps\Map004.txt
demo_jap\Maps\Map005.txt
demo_jap\Maps\Map006.txt
demo_jap\Maps\Map007.txt
demo_jap\Maps\Map008.txt
demo_jap\Maps\Map009.txt
demo_jap\Maps\Map010.txt
demo_jap\Maps\Map011.txt
demo_jap\Maps\Map012.txt
demo_jap\Maps\Map013.txt
demo_jap\Maps\Map014.txt
demo_jap\Maps\Map015.txt
demo_jap\Maps\Map016.txt
demo_jap\Maps\Map017.txt
demo_jap\Maps\Map018.txt
demo_jap\Maps\Map019.txt
demo_jap\Maps\Map020.txt
demo_jap\Maps\Map021.txt
demo_jap\Maps\Map022.txt
demo_jap\Maps\Map023.txt
demo_jap\Maps\Map024.txt
demo_jap\Maps\Map025.txt
nigger 200 
 199
demo_jap\Maps\Map026.txt
demo_jap\Maps\Map027.txt
demo_jap\Maps\Map028.txt
demo_jap\Maps\Map029.txt
demo_jap\Maps\Map030.txt
demo_jap\Maps\Map031.txt
demo_jap\Maps\Map032.txt
demo_jap\Maps\Map033.txt
demo_jap\Maps\Map034.txt
demo_jap\Maps\Map035.txt
demo_jap\Maps\Map036.txt
demo_jap\Maps\Map037.txt
demo_jap\Maps\Map038.txt
nigger 375 
 376
demo_jap\Maps\Ma

In [30]:
MAPS_ATTENSION = "translate_me_maps"
os.makedirs(f"{TRANSLATION_FOLDER}/{MAPS_FOLDER}/Map/Data", exist_ok=True)
os.makedirs(f"{TRANSLATION_FOLDER}/{MAPS_FOLDER}/Map2/Data", exist_ok=True)
os.makedirs(f"{TRANSLATION_FOLDER}/{MAPS_ATTENSION}/Map/Data", exist_ok=True)
os.makedirs(f"{TRANSLATION_FOLDER}/{MAPS_ATTENSION}/Map2/Data", exist_ok=True)


full_patched_maps = 0
total_maps = 0
with open("nigger.txt", 'w', encoding='utf-8') as file:
    file.write(str(vocabulary))
for item in Path(f"{FOLDER_TO_PATCH}/{MAPS_FOLDER}").glob('**/*.txt'):
    succesfully_patched = True
    text = item.read_text(encoding='utf-8')
    chunks = break_text_into_chunks(text)
    total_maps += 1
    for chunk in chunks:
        for piece in break_chunk(chunk[1]):
            if piece in simple_vocabulary:
                text = text.replace(piece, simple_vocabulary[piece])
            elif piece in vocabulary:
                text = text.replace(piece, vocabulary[piece])
            else:
                succesfully_patched = False

    if succesfully_patched:
        with open(str(item).replace(FOLDER_TO_PATCH, TRANSLATION_FOLDER), 'w', encoding="utf-8") as transition_file:
            transition_file.write(text)
        full_patched_maps += 1
    else:
        with open(str(item).replace(FOLDER_TO_PATCH, TRANSLATION_FOLDER).replace(MAPS_FOLDER,MAPS_ATTENSION), 'w', encoding="utf-8") as transition_file:
            transition_file.write(text)

print(f"Maps fully patched {full_patched_maps} / {total_maps}")

Maps fully patched 814 / 1846
